# Sandbox for testing FastSurfer model
- https://github.com/nikhil153/FastSurfer

In [1]:
import nibabel as nib
import numpy as np
import pandas as pd

def convert_mgh_to_nifti(mgh_file):
    nii_file = mgh_file.rsplit('.',1)[0] + '.nii.gz'
    mgh = nib.load(mgh_file)
    nib.save(mgh, nii_file)

def dice_coef(y_true, y_pred, smooth=1):
  intersection = np.sum(y_true * y_pred)
  union = np.sum(y_true) + np.sum(y_pred)
  dice = np.mean((2. * intersection + smooth)/(union + smooth))
  return dice

def get_multi_label_dice(label_list, y_true, y_pred):
    ''' Calculates label-wise dice score for a 3D segmentation volume
    '''
    dice_dict = {}
    for label in label_list:
        y_true_label = y_true == label
        y_pred_label = y_pred == label
        dice = dice_coef(y_true_label.ravel(), y_pred_label.ravel())
        dice_dict[label] = dice

    return pd.DataFrame(list(dice_dict.items()),columns=['label','dice'])

In [ ]:
data_dir = '/home/nikhil/projects/green_comp_neuro/FastSurfer/data/sub-0039/'
mgh_brain = data_dir + 'orig.mgz'

fs_seg_file = data_dir + 'aparc.DKTatlas+aseg.nii.gz'
deep_seg_file = data_dir + 'aparc.DKTatlas+aseg.deep.nii.gz'

fs_seg = nib.load(fs_seg_file).get_fdata()
deep_seg = nib.load(deep_seg_file).get_fdata()

unique_labels = set(fs_seg.ravel())

dice_df = get_multi_label_dice(unique_labels, fs_seg, deep_seg)